In [ ]:
%pip install tensorflow keras nltk scikit-learn transformers


In [ ]:
pip install --upgrade tensorflow

In [ ]:
pip install matplotlib

In [ ]:
pip uninstall transformers -y


In [ ]:
pip install google-generativeai

In [ ]:
pip install nltk

In [ ]:
pip install transformers tensorflow torch scikit-learn matplotlib

In [ ]:
pip cache purge

In [ ]:

pip install keras==2.13.1  transformers==4.33.0

In [ ]:
pip show tensorflow keras transformers

In [ ]:
pip install keras==3.0.5 transformers==4.38.1

In [ ]:
pip install tensorflow-addons


In [ ]:
#Another Version

In [ ]:
pip install sentence-transformers

In [9]:
import os
import json
import gc
import pickle
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM, Bidirectional, BatchNormalization, Layer, Add, LayerNormalization, MultiHeadAttention
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score

# ---- Custom Attention Layer ----
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(1,), initializer="zeros", trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        return K.sum(x * a, axis=1)

# ---- Load Dataset ----
with open("combined_dataset.json", "r") as f:
    parsed_data = [json.loads(line) for line in f if line.strip()]
contexts = [sample["Context"] for sample in parsed_data]

# ---- Load Mood Keywords ----
with open("mood_keywords.json", "r", encoding="utf-8") as f:
    mood_keywords = json.load(f)

def detect_mood(text):
    text = text.lower()
    for mood, keywords in mood_keywords.items():
        if any(keyword in text for keyword in keywords):
            return mood
    return None

X_texts, Y_labels = [], []
for context in contexts:
    mood = detect_mood(context)
    if mood:
        X_texts.append(context)
        Y_labels.append(mood)

# ---- Encode and Filter Labels ----
label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y_labels)
with open("mood_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

label_counts = Counter(Y_encoded)
valid_labels = {label for label, count in label_counts.items() if count > 1}
filtered = [(x, y) for x, y in zip(X_texts, Y_encoded) if y in valid_labels]
X_texts_filtered = [x for x, _ in filtered]
Y_filtered = [y for _, y in filtered]

# ---- SBERT Embedding ----
sbert = SentenceTransformer('paraphrase-MiniLM-L12-v2')
print("🔄 Generating SBERT embeddings...")
X_emb = sbert.encode(X_texts_filtered, show_progress_bar=True)

# ---- Resampling ----
min_samples = min(Counter(Y_filtered).values())
if min_samples < 2:
    print("⚠️ SMOTE skipped due to insufficient samples.")
    X_resampled, Y_resampled_encoded = X_emb, Y_filtered
else:
    smote_k = max(1, min(5, min_samples - 1))
    smote = SMOTE(k_neighbors=smote_k, random_state=42)
    X_smote, Y_smote = smote.fit_resample(X_emb, Y_filtered)
    smote_tomek = SMOTETomek(random_state=42)
    X_resampled, Y_resampled_encoded = smote_tomek.fit_resample(X_smote, Y_smote)

Y_resampled = to_categorical(Y_resampled_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(
    X_resampled, Y_resampled, test_size=0.25, stratify=np.argmax(Y_resampled, axis=1), random_state=42
)

X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)
Y_train_int = np.argmax(Y_train, axis=1)
Y_test_int = np.argmax(Y_test, axis=1)

class_weights = compute_class_weight("balanced", classes=np.unique(Y_train_int), y=Y_train_int)
class_weight_dict = dict(enumerate(class_weights))

# ---- Model Builder ----
def build_model(input_shape, num_classes):
    def transformer_block(inputs, num_heads=4, ff_dim=256, dropout=0.3):
        attention = MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(inputs, inputs)
        attention = Dropout(dropout)(attention)
        out1 = LayerNormalization(epsilon=1e-6)(Add()([inputs, attention]))
        ff = Dense(ff_dim, activation='relu')(out1)
        ff = Dense(inputs.shape[-1])(ff)
        ff = Dropout(dropout)(ff)
        return LayerNormalization(epsilon=1e-6)(Add()([out1, ff]))

    inputs = Input(shape=input_shape)
    x = transformer_block(inputs)
    x = transformer_block(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)
    x = AttentionLayer()(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ---- Train Model ----
gc.collect()
model = build_model(input_shape=X_train.shape[1:], num_classes=Y_resampled.shape[1])
early_stop = EarlyStopping(monitor="val_loss", patience=9, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.8, patience=7, min_lr=1e-6, verbose=1)

e:\AI ML Models\.venv\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


🔄 Generating SBERT embeddings...


Batches: 100%|██████████| 110/110 [00:59<00:00,  1.86it/s]
e:\AI ML Models\.venv\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
e:\AI ML Models\.venv\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
e:\AI ML Models\.venv\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
e:\AI ML Models\.venv\lib\site-p

In [ ]:
print("🚀 Training model...")
history = model.fit(
    X_train, Y_train_int,
    validation_data=(X_test, Y_test_int),
    epochs=20,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

test_loss, test_acc = model.evaluate(X_test, Y_test_int, verbose=1)
print(f"✅ Test Accuracy: {test_acc * 100:.2f}%")
model.save("mood_classifier_model.keras", save_format="keras")
print("💾 Model saved as mood_classifier_model.keras")

# ---- Plot Results ----
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epochs'); plt.ylabel('Accuracy'); plt.title('Accuracy'); plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epochs'); plt.ylabel('Loss'); plt.title('Loss'); plt.legend()
plt.tight_layout()
plt.show()

# ---- F1 Score ----
Y_pred_probs = model.predict(X_test, verbose=1)
Y_pred = np.argmax(Y_pred_probs, axis=1)
f1 = f1_score(Y_test_int, Y_pred, average="weighted")
print(f"🎯 Weighted F1 Score: {f1 * 100:.2f}%")

In [ ]:
import os
import json
import pickle
import numpy as np
from datetime import datetime
from sentence_transformers import SentenceTransformer
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K

# ---- Custom Attention Layer ----
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="normal", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(1,),
                                 initializer="zeros", trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        return K.sum(x * a, axis=1)

# ---- Load Model ----
try:
    model = load_model("mood_classifier_model.keras", custom_objects={"AttentionLayer": AttentionLayer})
    print("✅ Model loaded cleanly.")
except Exception as e:
    print(f"❌ Failed to load model: {e}")
    exit()

# ---- Load SBERT and Resources ----
sbert = SentenceTransformer("paraphrase-MiniLM-L12-v2")

with open("mood_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

with open("mood_keywords.json", "r", encoding="utf-8") as f:
    mood_keywords = json.load(f)

with open("extracted_questions.json", "r", encoding="utf-8") as f:
    question_data = json.load(f)

# ---- Logging ----
LOG_FILE = "chat_log.json"

def log_user_query(user_input, mood, question):
    entry = {
        "timestamp": datetime.now().isoformat(),
        "user_input": user_input,
        "predicted_mood": mood,
        "response_question": question
    }
    if os.path.exists(LOG_FILE):
        with open(LOG_FILE, "r", encoding="utf-8") as f:
            logs = json.load(f)
    else:
        logs = []
    logs.append(entry)
    with open(LOG_FILE, "w", encoding="utf-8") as f:
        json.dump(logs, f, indent=2)
    print("📝 Query logged.")

# ---- Mood Detection ----
def detect_mood_keywords(text):
    text = text.lower()
    for mood, keywords in mood_keywords.items():
        if any(keyword in text for keyword in keywords):
            return mood
    return None

def predict_mood(text):
    mood = detect_mood_keywords(text)
    if mood:
        print(f"📌 Detected mood from keywords: {mood}")
        return mood

    embedding = sbert.encode([text])
    embedding = np.expand_dims(embedding, axis=1)  # Shape: (1, 1, 384)
    prediction = model.predict(embedding, verbose=0)
    mood_index = np.argmax(prediction)
    mood_label = label_encoder.inverse_transform([mood_index])[0]
    print(f"🤖 Predicted mood from model: {mood_label}")
    return mood_label

# ---- Fetch Question ----
def fetch_question(mood):
    mood_sections = question_data.get(mood, [])
    all_questions = []

    for section in mood_sections:
        general = section.get("General", {})
        questions = general.get("questions", [])
        if isinstance(questions, list):
            all_questions.extend(questions)

    if not all_questions:
        return "I'm here to support you. Can you tell me more about how you're feeling?"
    return np.random.choice(all_questions)

# ---- Chatbot Main ----
def chatbot():
    print("🧠 ChatBot is ready. Type 'exit' to quit.\n")
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() in ["exit", "quit"]:
            print("👋 Take care. I'm here whenever you need support.")
            break

        mood = predict_mood(user_input)
        question = fetch_question(mood)
        print(f"MoodBot ({mood}): {question}\n")
        log_user_query(user_input, mood, question)

# ---- Run ----
if __name__ == "__main__":
    chatbot()

e:\AI ML Models\.venv\lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 122 variables whereas the saved optimizer has 110 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


❌ Failed to load model: A total of 2 objects could not be loaded. Example error message for object <LSTMCell name=lstm_cell, built=True>:

Layer 'lstm_cell' expected 3 variables, but received 0 variables during loading. Expected: ['kernel', 'recurrent_kernel', 'bias']

List of objects that could not be loaded:
[<LSTMCell name=lstm_cell, built=True>, <LSTMCell name=lstm_cell, built=True>]


e:\AI ML Models\.venv\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


🧠 ChatBot is ready. Type 'exit' to quit.

📌 Detected mood from keywords: Depression
MoodBot (Depression): I am not particularly discouraged about the future.

📝 Query logged.
🤖 Predicted mood from model: Mood_19
MoodBot (Mood_19): I'm here to support you. Can you tell me more about how you're feeling?

📝 Query logged.
👋 Take care. I'm here whenever you need support.


: 

In [11]:
import json
import nltk
from collections import defaultdict, Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Ensure NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words("english"))

# Load short mood dataset
with open("short_mood_dataset.json", "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f if line.strip()]

# Normalize and group contexts by mood
mood_to_contexts = defaultdict(list)
for sample in data:
    mood = sample.get("Mood", "").strip().title()
    context = sample.get("Context", "").strip().lower()
    if mood and context:
        mood_to_contexts[mood].append(context)

# Extract keywords per mood
mood_keywords = {}
for mood, contexts in mood_to_contexts.items():
    if len(contexts) < 2:
        continue  # skip rare moods
    all_text = " ".join(contexts)
    tokens = word_tokenize(all_text)
    cleaned = [w for w in tokens if w.isalpha() and w.lower() not in stop_words]
    top_keywords = [word.lower() for word, _ in Counter(cleaned).most_common(15)]
    if top_keywords:
        mood_keywords[mood] = top_keywords

# Save result
with open("mood_keywords.json", "w", encoding="utf-8") as f:
    json.dump(mood_keywords, f, indent=2, ensure_ascii=False)

print(f"✅ mood_keywords.json generated with {len(mood_keywords)} moods.")


# Load the old mood_keywords.json (with Mood_0, Mood_1... keys)
with open("mood_keywords.json", "r", encoding="utf-8") as f:
    mood_keywords = json.load(f)

# Define mapping from generic Mood_# labels to real mood names
mood_map = {
    "Mood_0": "Abuse",
    "Mood_1": "Sexual Issues",
    "Mood_2": "Erectile Dysfunction",
    "Mood_3": "Relationship Issues",
    "Mood_4": "Depression",
    "Mood_5": "Child Custody",
    "Mood_6": "Parental Issues",
    "Mood_7": "Infidelity",
    "Mood_8": "Marriage Problems",
    "Mood_9": "Motherhood",
    "Mood_10": "Gender Identity",
    "Mood_11": "Therapist Issues",
    "Mood_12": "Therapy Anxiety",
    "Mood_13": "Marital Issues",
    "Mood_14": "Unrequited Love",
    "Mood_15": "Therapy Nervousness",
    "Mood_16": "Health Trauma",
    "Mood_17": "PTSD",
    "Mood_18": "Alcoholism",
    "Mood_19": "Breakups",
    "Mood_20": "Sexless Marriage",
    "Mood_21": "Hearing Voices",
    "Mood_22": "Eating Disorders",
    "Mood_23": "Suicidal Thoughts",
    "Mood_24": "Panic Attacks",
    "Mood_25": "Toxic Relationships",
    "Mood_26": "Anxiety And Depression",
    "Mood_27": "Daughter Stress",
    "Mood_28": "Work And Family Stress",
    "Mood_29": "School Stress"
}

# Create a new dict with renamed keys
renamed_keywords = {}
for mood_id, keywords in mood_keywords.items():
    readable_label = mood_map.get(mood_id, mood_id)  # fallback to original key if no match
    renamed_keywords[readable_label] = keywords

# Save the updated file
with open("mood_keywords.json", "w", encoding="utf-8") as f:
    json.dump(renamed_keywords, f, indent=2, ensure_ascii=False)

print(f"✅ mood_keywords.json updated and saved with {len(renamed_keywords)} mood labels.")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\soham\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\soham\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✅ mood_keywords.json generated with 30 moods.
✅ mood_keywords.json updated and saved with 30 mood labels.
